#### <font color=blue>SVD - NewIndex</font>

[UCSD](https://cseweb.ucsd.edu/~jmcauley/datasets.html#clothing_fit) dataset on clothing fit

**Item-based CF**:
- <font color=red>items usually don't change much</font> so this approach can be computed offline
- uses patterns of users who browsed the same item as active user
- <font color=blue>"Users who liked this item also liked ..."</font>

**User-based CF**:
- use k-nearest neighbors to find <font color=red>clusters of similar users based on common item ratings</font>
- then make predictions using the <font color=red>average rating of top k-nearest neighbors</font>

- <font color=blue>"Users who are similar to you also liked ..."</font>

In [1]:
## importing modules 

import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'retina'
plt.style.use('fivethirtyeight')
import itertools

# surprise modules
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import SlopeOne
from surprise import CoClustering
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise.accuracy import rmse

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

In [2]:
## read csv file

df = pd.read_csv('renttherunway_reduced25_new.csv')

print(df.shape)
df.head()

(1874, 26)


,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,review_summary,review_text,size,user_id,weight,band_size,cup_size,feet,inches,height_inch,weight_lbs,year,month,day,item_id_size,item_id_new
0,32.0,hourglass,32c,dress,fit,"5' 3""",143094,5.0,party,2015-04-07,glamorous like a sexy disco ball!,few friends and i wore this dress and went to ...,8,610914,140lbs,32.0,c,5,3,63,140.0,2015,4,7,14309408,751.0
1,38.0,straight & narrow,34c,dress,fit,"5' 8""",143094,2.0,vacation,2012-11-25,looked like a showstopper but i was itching al...,"the dress fit great, but the sequins did not l...",8,53519,135lbs,34.0,c,5,8,68,135.0,2012,11,25,14309408,751.0
2,37.0,athletic,36c,dress,fit,"5' 4""",143094,5.0,party,2014-04-15,perfect dress for my bachelorette party!,fit perfectly without being too tight. materia...,8,470639,138lbs,36.0,c,5,4,64,138.0,2014,4,15,14309408,751.0
3,47.0,athletic,34b,dress,fit,"5' 5""",145417,5.0,other,2014-01-15,great for work-to-dinner party,form fitting and flattering. this dress was c...,8,276186,120lbs,34.0,b,5,5,65,120.0,2014,1,15,14541708,804.0
4,36.0,pear,34c,dress,fit,"5' 7""",145417,4.0,party,2016-01-20,charity event,this dress is a great color and is a nubby mat...,8,748397,125lbs,34.0,c,5,7,67,125.0,2016,1,20,14541708,804.0


In [3]:
## Ratings distribution

df['rating'].value_counts()

5.0    1220
4.0     495
3.0     123
2.0      28
1.0       8
Name: rating, dtype: int64

In [4]:
df['user_id'].nunique()

755

In [5]:
df['item_id_new'].nunique()

797

<div class='alert alert-danger'>
#### Using the Surprise module
- load dataset from pandas dataframe using `Dataset.load_from_df`

#### Algorithms to compare:
- `NormalPredictor`: predicts random rating based on training set which is assumed to be normally distributed
- `BaselineOnly`: predicts the baseline estimate for a given user and given item
- `Slope One`: factors in items that a user liked separately from items that a user disliked (by computing the average difference between ratings of one item and another for users who rated both)
- `Co-clustering`: simultaneous clustering of users and items

**Neighborhood collaborative filtering (CF)**:
- `KNNBasic`: basic nearest neighbors CF algorithm
- `KNNWithMeans`: takes into account the mean ratings of each user
- `KNNWithZScore`: takes into account the z-score normalization of each user
- `KNNBaseline`: takes into account a baseline rating

**Matrix factorization CF**:
- `SVD`: singular value decomposition is equivalent to [Probabilistic Matrix Factorization, PMF](http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf) if baselines are not used
- `SVDpp or SVD++`: takes into account implicit ratings
- `NMF`: non-negative matrix factorization

#### Model selection:
- manually split data into training set and test set (80-20)
- use `cross_validate` to select model with lowest RSME score
- use `GridSearchCV` to tune hyperparameters for ratings prediction

In [6]:
## create reader and load specific columns from dataframe into surprise

reader = Reader(rating_scale=(1,5))

data = Dataset.load_from_df(df[['user_id','item_id_new','rating']], reader)

data1 = Dataset.load_from_df(df[['user_id','item_id_new','rating']], reader)

<div class='alert alert-warning'>
#### [manually split data](https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation) into training set and test set

In [7]:
raw_ratings = data.raw_ratings

random.shuffle(raw_ratings)

# set training set as 80% of the data, test set as 20%
threshold = int(0.8 * len(raw_ratings))

train_raw_ratings = raw_ratings[:threshold]
test_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = train_raw_ratings

<div class='alert alert-warning'>
#### compare algorithms using cross validation and pre-defined training set (80%)
- using metrics: `RMSE`, `MAE`

In [8]:
comparison = []

algos = [NormalPredictor(), BaselineOnly(), SlopeOne(), CoClustering(),\
         KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline(),\
         SVD(), SVDpp(), NMF()]

for algorithm in algos:
    score = cross_validate(algorithm, data, measures=['RMSE','MAE'],\
                             cv=5, verbose=False)
    
    result = pd.DataFrame.from_dict(score).mean(axis=0)
    result = result.append(pd.Series([str(algorithm).split(' ')[0].\
                                      split('.')[-1]],\
                                     index=['Algorithm']))
    comparison.append(result)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

In [9]:
pd.DataFrame(comparison).set_index('Algorithm').\
sort_values('test_rmse',ascending=True)

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
BaselineOnly,0.715041,0.582926,0.004834,0.001858
SVD,0.719147,0.579253,0.107441,0.031143
KNNBaseline,0.720838,0.583231,0.017434,0.003763
KNNBasic,0.722276,0.590472,0.012402,0.006514
SVDpp,0.724872,0.580426,0.222084,0.004703
KNNWithZScore,0.793552,0.572186,0.047598,0.005034
KNNWithMeans,0.810397,0.585047,0.022079,0.005146
SlopeOne,0.813881,0.580489,0.016755,0.002475
CoClustering,0.861428,0.626038,0.167917,0.002850


<div class='alert alert-danger'>
#### use the 4 best algorithms to get predictions for existing testset ratings: 
- `BaselineOnly()`, `SVDpp()`, `SVD()`, `KNNWithZScore()`
- create dataframe for predictions of existing testset ratings
- note: lower RMSE/MAE is better

In [10]:
def get_items(user_id):
    """ returns the number of items rated by a given user
    args: 
      uid: the 'raw' id of the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(user_id)])
            # to_inner_uid() converts raw user id to inner id

    except ValueError: # user was not part of the trainset
        return 0
    
def get_users(item_id):
    """ returns number of users that have rated a given item
    args:
      iid: the raw id of the item
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(item_id)])
            # to_inner_iid() converts raw item id to inner id
    except ValueError:
        return 0

#### use GridSearch to find the best parameters for SVDpp and SVD

```Python
param_grid = {'n_epochs': [15,25],'lr_all': [0.003, 0.005, 0.007],\
              'reg_all': [0.02, 0.06, 0.1]}

gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'],\
                  cv=3, joblib_verbose=100)
gs.fit(data1)
print('best parameters for SVDpp: ', gs.best_params['rmse'])

gs1 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'],\
                   cv=3, joblib_verbose=100)
gs1.fit(data1)
print('best parameters for SVD: ', gs1.best_params['rmse'])
```
- best parameters for SVDpp: {**'n_epochs': 15, 'lr_all': 0.005, 'reg_all': 0.02**}
- best parameters for SVD: {**'n_epochs': 15, 'lr_all': 0.007, 'reg_all': 0.1**}

### predict using SVD

In [11]:
# use GridSearchCV to find best parameters for SVD

trainset = data.build_full_trainset()

algo3 = SVDpp(n_epochs=15,lr_all=0.007,reg_all=0.1)
algo3.fit(trainset)

predictions3_biased = algo3.test(trainset.build_testset())
print('Biased accuracy on training set (SVD): ', end='   ')
accuracy.rmse(predictions3_biased)

###########################################################
testset = data.construct_testset(test_raw_ratings)
predictions3_unbiased = algo3.test(testset)
print('Unbiased accuracy on test set (SVD): ', end='   ')
accuracy.rmse(predictions3_unbiased)

Biased accuracy on training set (SVD):    RMSE: 0.5595
Unbiased accuracy on test set (SVD):    RMSE: 0.7235


0.7235305312111795

In [12]:
# checking that the train-test split is indeed 80-20

print('length of the train set: ',trainset.n_ratings)
print('length of the test set: ',len(testset))

length of the train set:  1499
length of the test set:  375


In [13]:
## ## top 10 best and worst predictions for the testset using SVD

df_SVD = pd.DataFrame(predictions3_unbiased, columns=['uid','iid','actual rating',\
                                                        'estimated rating','details'])
df_SVD['number of items for target user'] = df_SVD['uid'].apply(get_items)
df_SVD['number of users for target item'] = df_SVD['iid'].apply(get_users)
df_SVD['error'] = abs(df_SVD['estimated rating'] - df_SVD['actual rating'])

Best_predictions_SVD = df_SVD.sort_values(by='error')[:10]
Best_predictions_SVD

,uid,iid,actual rating,estimated rating,details,number of items for target user,number of users for target item,error
81,327294,19753.0,4.0,3.982946,{'was_impossible': False},6,0,0.017054
46,777615,27119.0,5.0,4.942945,{'was_impossible': False},4,7,0.057055
118,888739,29231.0,4.0,4.058714,{'was_impossible': False},1,3,0.058714
166,778579,16551.0,4.0,4.116548,{'was_impossible': False},4,2,0.116548
214,359031,19077.0,5.0,4.838279,{'was_impossible': False},5,1,0.161721
112,290985,65.0,5.0,4.820019,{'was_impossible': False},6,3,0.179981
263,952171,11966.0,5.0,4.813747,{'was_impossible': False},3,4,0.186253
92,635670,9605.0,5.0,4.809996,{'was_impossible': False},3,3,0.190004
99,172691,11507.0,5.0,4.790311,{'was_impossible': False},2,8,0.209689
47,279715,1276.0,4.0,4.218420,{'was_impossible': False},3,6,0.218420


In [14]:
Worst_predictions_SVD = df_SVD.sort_values(by='error')[-10:]
Worst_predictions_SVD

,uid,iid,actual rating,estimated rating,details,number of items for target user,number of users for target item,error
358,777877,23199.0,3.0,4.733560,{'was_impossible': False},3,5,1.733560
75,586467,12286.0,3.0,4.764966,{'was_impossible': False},3,3,1.764966
292,457730,23414.0,2.0,4.406571,{'was_impossible': False},0,1,2.406571
264,96369,16162.0,2.0,4.423015,{'was_impossible': False},3,2,2.423015
269,970467,11965.0,2.0,4.456188,{'was_impossible': False},3,1,2.456188
0,691468,979.0,2.0,4.508609,{'was_impossible': False},23,0,2.508609
322,145029,15858.0,2.0,4.513615,{'was_impossible': False},1,7,2.513615
356,53519,751.0,2.0,4.621289,{'was_impossible': False},1,2,2.621289
86,31844,11449.0,2.0,4.709707,{'was_impossible': False},1,2,2.709707
196,606519,15257.0,1.0,4.399935,{'was_impossible': False},1,1,3.399935


<div class='alert alert-warning'>
#### get top-N recommendations for each user

- first train each algorithm on the whole dataset
- then predict all the **ratings for user-item pairs that are not in** the dataset
- retrieve top-N predictions for each user

In [15]:
## using get_top_n function from surprise library

from collections import defaultdict

def get_top_n(predictions, n):
    ''' 
    Return the top-N recommendation for each user from set of predictions.
        
    Args:
    predictions(list of Prediction objects): The list of predictions,
        as returned by the test method of an algorithm.
    n(int): The number of recommendation to output for each user. 
    Default is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of
        tuples: [(raw item id, rating estimation), ...] of size n.
    '''
        
    # First map the predictions to each user.
    top_n = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the 
        # k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


def get_categ(itemID):
    ''' 
    Returns item category for a given itemID
    '''
    category = df[df['item_id_new']==itemID]['category'].values[0]
    
    return category


def get_existing(userID):
    ''' 
    Returns list of at most 10 random items that user has rented
    '''
    existing = df[df['user_id']==userID]['item_id_new']
    exdict = {}
    if len(existing) < 10:
        exist = list(df[df['user_id']==userID]['item_id_new'].\
                     sample(n=len(existing), random_state=1))
        exdict[userID] = [(a, get_categ(a)) for a in exist]
    else:
        exist = list(df[df['user_id']==userID]['item_id_new'].\
                     sample(n=10, random_state=1))
        exdict[userID] = [(a, get_categ(a)) for a in exist]
    return exdict

In [16]:
## train SVD algorithm on the whole dataset

trainset = data1.build_full_trainset()

algo = SVDpp(n_epochs=15,lr_all=0.007,reg_all=0.1)
algo.fit(trainset)

## predict all ratings (for user-item pairs) that are not in the train set
testset = trainset.build_anti_testset()
predictions_SVD = algo.test(testset)
accuracy.rmse(predictions_SVD)

topNPredicted_SVD = get_top_n(predictions_SVD, 10)

SVD_recc_dict = {}
SVD_exist_dict = {}

## print existing and recommended items for each user:
i=0
for uid, user_ratings in topNPredicted_SVD.items():
    exitem = get_existing(uid)
    
    print("Random 10 rented items for user {0}: {1}".\
          format(uid, list(a for a in exitem.values())[0]))
    SVD_exist_dict[uid] = list(a for a in exitem.values())[0]
    print()
    
    SVD_recc_dict[uid] = [(iid,get_categ(iid)) for (iid,_)in user_ratings]
    print('Top 10 recommended new items for user {0}: {1}'.\
          format(uid, [(iid,get_categ(iid)) for (iid,_) in user_ratings]))
    print('-------------------------------------------------------------')
    
    i+=1
print('Number of recommended item sets (SVD):', i)

RMSE: 0.1481
Random 10 rented items for user 610914: [(751.0, 'dress')]

Top 10 recommended new items for user 610914: [(11966.0, 'dress'), (27357.0, 'blazer'), (9738.0, 'dress'), (544.0, 'sheath'), (7637.0, 'dress'), (524.0, 'dress'), (17925.0, 'sheath'), (1375.0, 'dress'), (27119.0, 'jumpsuit'), (986.0, 'gown')]
-------------------------------------------------------------
Random 10 rented items for user 53519: [(751.0, 'dress'), (8003.0, 'dress')]

Top 10 recommended new items for user 53519: [(13679.0, 'dress'), (27762.0, 'jumpsuit'), (4816.0, 'sheath'), (11966.0, 'dress'), (10883.0, 'sheath'), (4779.0, 'dress'), (14158.0, 'shift'), (544.0, 'sheath'), (29294.0, 'romper'), (1375.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 470639: [(414.0, 'gown'), (528.0, 'dress'), (22588.0, 'dress'), (751.0, 'dress'), (878.0, 'dress')]

Top 10 recommended new items for user 470639: [(4816.0, 'sheath'), (27119.0, 'jumpsuit'), (23912.0, '

Top 10 recommended new items for user 648369: [(11507.0, 'dress'), (4816.0, 'sheath'), (11966.0, 'dress'), (171.0, 'dress'), (27357.0, 'blazer'), (14597.0, 'dress'), (12154.0, 'gown'), (528.0, 'dress'), (480.0, 'dress'), (544.0, 'sheath')]
-------------------------------------------------------------
Random 10 rented items for user 339506: [(170.0, 'dress'), (710.0, 'dress'), (2560.0, 'maxi')]

Top 10 recommended new items for user 339506: [(2171.0, 'dress'), (27119.0, 'jumpsuit'), (7756.0, 'dress'), (11966.0, 'dress'), (8070.0, 'dress'), (17925.0, 'sheath'), (6979.0, 'dress'), (536.0, 'sheath'), (3798.0, 'dress'), (19753.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 571227: [(170.0, 'dress')]

Top 10 recommended new items for user 571227: [(17879.0, 'dress'), (1352.0, 'sheath'), (23304.0, 'sheath'), (7756.0, 'dress'), (65.0, 'dress'), (10883.0, 'sheath'), (1094.0, 'dress'), (556.0, 'gown'), (1163.0, 'dress'), (11371.0, 'dres

Top 10 recommended new items for user 822475: [(19306.0, 'gown'), (2171.0, 'dress'), (8070.0, 'dress'), (212.0, 'gown'), (3798.0, 'dress'), (11966.0, 'dress'), (27119.0, 'jumpsuit'), (11122.0, 'dress'), (528.0, 'dress'), (65.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 402094: [(6133.0, 'dress'), (28929.0, 'jumpsuit'), (1714.0, 'dress'), (1592.0, 'dress'), (1388.0, 'dress'), (1405.0, 'dress'), (2534.0, 'dress')]

Top 10 recommended new items for user 402094: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (17925.0, 'sheath'), (4816.0, 'sheath'), (17879.0, 'dress'), (20337.0, 'dress'), (11966.0, 'dress'), (544.0, 'sheath'), (23912.0, 'legging'), (7326.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 434928: [(4779.0, 'dress'), (1074.0, 'dress')]

Top 10 recommended new items for user 434928: [(23303.0, 'sheath'), (23304.0, 'sheath'), (17879.0, 'dress'), (556.0, 'gown'), (1

Random 10 rented items for user 835431: [(12080.0, 'dress'), (7756.0, 'dress'), (11449.0, 'dress'), (6979.0, 'dress')]

Top 10 recommended new items for user 835431: [(12630.0, 'dress'), (11966.0, 'dress'), (11507.0, 'dress'), (757.0, 'sheath'), (9738.0, 'dress'), (4816.0, 'sheath'), (27357.0, 'blazer'), (21013.0, 'sheath'), (2171.0, 'dress'), (27119.0, 'jumpsuit')]
-------------------------------------------------------------
Random 10 rented items for user 31844: [(11449.0, 'dress'), (7637.0, 'dress')]

Top 10 recommended new items for user 31844: [(27119.0, 'jumpsuit'), (27357.0, 'blazer'), (9499.0, 'dress'), (11966.0, 'dress'), (23912.0, 'legging'), (11507.0, 'dress'), (24585.0, 'jumpsuit'), (14158.0, 'shift'), (212.0, 'gown'), (27762.0, 'jumpsuit')]
-------------------------------------------------------------
Random 10 rented items for user 134802: [(21843.0, 'dress'), (13679.0, 'dress'), (22933.0, 'dress'), (7683.0, 'dress')]

Top 10 recommended new items for user 134802: [(1263

Top 10 recommended new items for user 517738: [(27119.0, 'jumpsuit'), (27762.0, 'jumpsuit'), (11966.0, 'dress'), (17787.0, 'sheath'), (9605.0, 'dress'), (1087.0, 'dress'), (11507.0, 'dress'), (22892.0, 'sheath'), (39.0, 'gown'), (1277.0, 'mini')]
-------------------------------------------------------------
Random 10 rented items for user 51748: [(5781.0, 'dress')]

Top 10 recommended new items for user 51748: [(27119.0, 'jumpsuit'), (9499.0, 'dress'), (1094.0, 'dress'), (769.0, 'dress'), (975.0, 'sheath'), (27357.0, 'blazer'), (39.0, 'gown'), (11507.0, 'dress'), (29042.0, 'jacket'), (11966.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 540884: [(8852.0, 'dress'), (4780.0, 'dress'), (16830.0, 'dress')]

Top 10 recommended new items for user 540884: [(27357.0, 'blazer'), (10883.0, 'sheath'), (528.0, 'dress'), (22127.0, 'gown'), (19306.0, 'gown'), (2171.0, 'dress'), (19770.0, 'dress'), (12385.0, 'dress'), (27119.0, 'jumpsuit'), 

...

Top 10 recommended new items for user 523001: [(27119.0, 'jumpsuit'), (4816.0, 'sheath'), (27762.0, 'jumpsuit'), (8003.0, 'dress'), (17879.0, 'dress'), (17786.0, 'sheath'), (11966.0, 'dress'), (526.0, 'dress'), (23303.0, 'sheath'), (65.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 868765: [(14597.0, 'dress'), (1086.0, 'dress'), (19500.0, 'dress')]

Top 10 recommended new items for user 868765: [(27119.0, 'jumpsuit'), (11966.0, 'dress'), (4816.0, 'sheath'), (8003.0, 'dress'), (47.0, 'gown'), (544.0, 'sheath'), (27762.0, 'jumpsuit'), (24585.0, 'jumpsuit'), (19475.0, 'dress'), (17277.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 624501: [(4818.0, 'sheath')]

Top 10 recommended new items for user 624501: [(1087.0, 'dress'), (12151.0, 'gown'), (8536.0, 'dress'), (21013.0, 'sheath'), (27119.0, 'jumpsuit'), (23303.0, 'sheath'), (11507.0, 'dress'), (528.0, 'dress'), (556.0,

Random 10 rented items for user 335054: [(5783.0, 'dress'), (1541.0, 'dress'), (1392.0, 'dress')]

Top 10 recommended new items for user 335054: [(10883.0, 'sheath'), (17879.0, 'dress'), (17141.0, 'sheath'), (7637.0, 'dress'), (8003.0, 'dress'), (300.0, 'gown'), (537.0, 'sheath'), (7328.0, 'dress'), (19306.0, 'gown'), (8094.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 814687: [(521.0, 'shift')]

Top 10 recommended new items for user 814687: [(11122.0, 'dress'), (1087.0, 'dress'), (27119.0, 'jumpsuit'), (11966.0, 'dress'), (4816.0, 'sheath'), (29558.0, 'jumpsuit'), (212.0, 'gown'), (17787.0, 'sheath'), (17925.0, 'sheath'), (11507.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 511128: [(5976.0, 'print'), (205.0, 'dress')]

Top 10 recommended new items for user 511128: [(11507.0, 'dress'), (27119.0, 'jumpsuit'), (83.0, 'dress'), (17879.0, 'dress'), (526.0, 'dress'), (7

Random 10 rented items for user 623315: [(15429.0, 'dress')]

Top 10 recommended new items for user 623315: [(17879.0, 'dress'), (19306.0, 'gown'), (12630.0, 'dress'), (36.0, 'gown'), (9329.0, 'dress'), (4817.0, 'sheath'), (4816.0, 'sheath'), (19770.0, 'dress'), (17786.0, 'sheath'), (288.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 585785: [(26855.0, 'jumpsuit')]

Top 10 recommended new items for user 585785: [(39.0, 'gown'), (4816.0, 'sheath'), (1094.0, 'dress'), (11966.0, 'dress'), (9499.0, 'dress'), (23412.0, 'shift'), (972.0, 'sheath'), (14429.0, 'sheath'), (29042.0, 'jacket'), (17786.0, 'sheath')]
-------------------------------------------------------------
Random 10 rented items for user 392871: [(479.0, 'dress'), (19500.0, 'dress')]

Top 10 recommended new items for user 392871: [(544.0, 'sheath'), (27119.0, 'jumpsuit'), (4816.0, 'sheath'), (11507.0, 'dress'), (11966.0, 'dress'), (12630.0, 'dress'), (17925.0, 'sheath

Random 10 rented items for user 113893: [(20189.0, 'shift')]

Top 10 recommended new items for user 113893: [(27119.0, 'jumpsuit'), (17879.0, 'dress'), (27762.0, 'jumpsuit'), (23303.0, 'sheath'), (3798.0, 'dress'), (1277.0, 'mini'), (7756.0, 'dress'), (11507.0, 'dress'), (1450.0, 'dress'), (11966.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 53764: [(4602.0, 'gown')]

Top 10 recommended new items for user 53764: [(27119.0, 'jumpsuit'), (39.0, 'gown'), (11741.0, 'dress'), (300.0, 'gown'), (8391.0, 'sheath'), (19306.0, 'gown'), (23412.0, 'shift'), (17879.0, 'dress'), (2171.0, 'dress'), (4816.0, 'sheath')]
-------------------------------------------------------------
Random 10 rented items for user 751304: [(9329.0, 'dress'), (275.0, 'dress'), (742.0, 'gown')]

Top 10 recommended new items for user 751304: [(27119.0, 'jumpsuit'), (1450.0, 'dress'), (4816.0, 'sheath'), (17879.0, 'dress'), (23303.0, 'sheath'), (11507.0, 'dress'), 

Top 10 recommended new items for user 293159: [(17879.0, 'dress'), (11507.0, 'dress'), (27119.0, 'jumpsuit'), (24583.0, 'jumpsuit'), (83.0, 'dress'), (10883.0, 'sheath'), (28976.0, 'romper'), (23303.0, 'sheath'), (1614.0, 'maxi'), (23912.0, 'legging')]
-------------------------------------------------------------
Random 10 rented items for user 601986: [(18073.0, 'gown')]

Top 10 recommended new items for user 601986: [(27357.0, 'blazer'), (11966.0, 'dress'), (8094.0, 'dress'), (11507.0, 'dress'), (17426.0, 'sheath'), (769.0, 'dress'), (16828.0, 'dress'), (16009.0, 'maxi'), (14158.0, 'shift'), (23912.0, 'legging')]
-------------------------------------------------------------
Random 10 rented items for user 230384: [(20718.0, 'dress')]

Top 10 recommended new items for user 230384: [(1087.0, 'dress'), (11507.0, 'dress'), (20337.0, 'dress'), (17787.0, 'sheath'), (19484.0, 'dress'), (212.0, 'gown'), (27119.0, 'jumpsuit'), (13679.0, 'dress'), (480.0, 'dress'), (1135.0, 'gown')]
----------

In [17]:
## compare results for SVD

df_exist = pd.DataFrame.from_dict(SVD_exist_dict,orient='index',\
                                  columns=['rent1','rent2','rent3',\
                                           'rent4','rent5','rent6',\
                                           'rent7','rent8','rent9',\
                                           'rent10']).reset_index()

df_rec = pd.DataFrame.from_dict(SVD_recc_dict, orient='index',\
                                columns=['rec1','rec2','rec3','rec4',\
                                         'rec5','rec6','rec7','rec8',\
                                         'rec9','rec10']).reset_index()

df_compare = pd.concat([df_exist, df_rec], axis=1).\
rename(columns={'index':'item ID'})

df_compare

,item ID,rent1,rent2,rent3,rent4,rent5,rent6,rent7,rent8,rent9,rent10,item ID,rec1,rec2,rec3,rec4,rec5,rec6,rec7,rec8,rec9,rec10
0,610914,"(751.0, dress)",None,None,None,None,None,None,None,None,None,610914,"(11966.0, dress)","(27357.0, blazer)","(9738.0, dress)","(544.0, sheath)","(7637.0, dress)","(524.0, dress)","(17925.0, sheath)","(1375.0, dress)","(27119.0, jumpsuit)","(986.0, gown)"
1,53519,"(751.0, dress)","(8003.0, dress)",None,None,None,None,None,None,None,None,53519,"(13679.0, dress)","(27762.0, jumpsuit)","(4816.0, sheath)","(11966.0, dress)","(10883.0, sheath)","(4779.0, dress)","(14158.0, shift)","(544.0, sheath)","(29294.0, romper)","(1375.0, dress)"
2,470639,"(414.0, gown)","(528.0, dress)","(22588.0, dress)","(751.0, dress)","(878.0, dress)",None,None,None,None,None,470639,"(4816.0, sheath)","(27119.0, jumpsuit)","(23912.0, legging)","(12630.0, dress)","(11507.0, dress)","(11966.0, dress)","(544.0, sheath)","(335.0, gown)","(20712.0, sheath)","(4602.0, gown)"
3,276186,"(804.0, dress)",None,None,None,None,None,None,None,None,None,276186,"(27119.0, jumpsuit)","(27762.0, jumpsuit)","(2171.0, dress)","(480.0, dress)","(11741.0, dress)","(8536.0, dress)","(11966.0, dress)","(20713.0, sheath)","(65.0, dress)","(17879.0, dress)"
4,748397,"(17879.0, dress)","(800.0, sheath)","(5093.0, dress)","(804.0, dress)","(26856.0, jumpsuit)",None,None,None,None,None,748397,"(4816.0, sheath)","(27357.0, blazer)","(27119.0, jumpsuit)","(29294.0, romper)","(11507.0, dress)","(27068.0, skirt)","(544.0, sheath)","(11741.0, dress)","(17426.0, sheath)","(8094.0, dress)"
5,856829,"(9499.0, dress)","(1276.0, mini)","(804.0, dress)","(545.0, sheath)",None,None,None,None,None,None,856829,"(4816.0, sheath)","(11741.0, dress)","(11966.0, dress)","(39.0, gown)","(10883.0, sheath)","(11507.0, dress)","(27762.0, jumpsuit)","(12630.0, dress)","(544.0, sheath)","(1277.0, mini)"
6,990544,"(804.0, dress)",None,None,None,None,None,None,None,None,None,990544,"(22892.0, sheath)","(27762.0, jumpsuit)","(39.0, gown)","(1087.0, dress)","(27119.0, jumpsuit)","(29042.0, jacket)","(11966.0, dress)","(4816.0, sheath)","(11507.0, dress)","(8536.0, dress)"
7,214108,"(8157.0, dress)",None,None,None,None,None,None,None,None,None,214108,"(29294.0, romper)","(13679.0, dress)","(4816.0, sheath)","(1094.0, dress)","(1375.0, dress)","(27068.0, skirt)","(16009.0, maxi)","(4779.0, dress)","(407.0, gown)","(1176.0, dress)"
8,503301,"(524.0, dress)","(1387.0, dress)","(8157.0, dress)","(1450.0, dress)",None,None,None,None,None,None,503301,"(4816.0, sheath)","(779.0, gown)","(28598.0, blazer)","(300.0, gown)","(13679.0, dress)","(975.0, sheath)","(11507.0, dress)","(27357.0, blazer)","(10413.0, dress)","(544.0, sheath)"
9,574571,"(16499.0, dress)","(1005.0, mini)","(8157.0, dress)","(1602.0, dress)",None,None,None,None,None,None,574571,"(17925.0, sheath)","(17879.0, dress)","(19306.0, gown)","(65.0, dress)","(779.0, gown)","(23304.0, sheath)","(23412.0, shift)","(10883.0, sheath)","(11966.0, dress)","(8536.0, dress)"


<div class='alert alert-danger'>
**Other metrics to evaluate top N recommendations**:

[Source 1](https://gab41.lab41.org/recommender-systems-its-not-all-about-the-accuracy-562c7dceeaff) ; 
[Source 2](https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093) ; 
[rec metrics Python library (from source 2)](https://github.com/statisticianinstilettos/recmetrics)

<div class='alert alert-warning'>
**Coverage**:
- <font color=red>what % of user-item space can be recommended?</font>
- **User coverage**: measures the percentage of users with at least one "good" recommendation (rating above a certain threshold). A higher value is better. (numUsers = trainset.n_users)
- **Item coverage**: measures the percentage of recommended items (with a rating above a certain threshold). A higher value is better. (numItems = trainset.n_items)

In [18]:
## % of items in train set that model is able to recommend on a test set

def Coverage(predicted, length):
    """
    Computes the coverage for a list of recommendations
    Parameters:
    ----------
    predicted : a dictionary of lists (of item-rating pairs)
    
    length: integer
        The number of unique items in the whole training set
    
    Returns:
    ----------
    The coverage of the recommendations as a percent 
    rounded to 2 decimal places.
    """
    predicted_items = [iid for uid, user_ratings in predicted.items()\
                       for (iid, _) in user_ratings]
    unique_pred_items = len(set(predicted_items))
    coverage = round(unique_pred_items/(length*1.0)*100,2)/100
    return coverage

## for the top N recommendations, the train set is the whole dataset

# print('Item coverage using BaselineOnly: {:.2%}'.\
#       format(Coverage(topNPredicted_BsO,len(df))))

# print('Item coverage using SVDpp: {:.2%}'.\
#       format(Coverage(topNPredicted_SVDpp,len(df))))

print('Item coverage using SVD: {:.2%}'.\
      format(Coverage(topNPredicted_SVD,len(df))))

# print('Item coverage using KNNWithZScore: {:.2%}'.\
#       format(Coverage(topNPredicted_KNNZ,len(df))))

Item coverage using SVD: 16.76%


In [19]:
def UserCoverage(topNPredicted, numUsers, min_rating=4.0):
    hits = 0
    for user in topNPredicted.keys():
        hit = False
        for user_ratings in topNPredicted[user]:
            if (user_ratings[1] >= min_rating):
                hit = True
                break
        if (hit):
            hits += 1

    return hits / numUsers

# print('User coverage using BaselineOnly: {:.2%}'.\
#       format(UserCoverage(topNPredicted_BsO, len(df))))

# print('User coverage using SVDpp: {:.2%}'.\
#       format(UserCoverage(topNPredicted_SVDpp, len(df))))

print('User coverage using SVD: {:.2%}'.\
      format(UserCoverage(topNPredicted_SVD, len(df))))

# print('User coverage using KNNWithZScore: {:.2%}'.\
#       format(UserCoverage(topNPredicted_KNNZ, len(df))))

User coverage using SVD: 40.29%


<div class='alert alert-warning'>
**Novelty**:
- <font color=red>how surprising are the recommendations?</font>
- measures how many unknown recommended items are to a user (serendipity vs popularity)
- <font color=red>popular items: must have the best rating of 5</font>
- higher novelty value = less popular items being recommended

In [20]:
df_cleaned = pd.read_csv('renttherunway_cleaned_new.csv')
print(df_cleaned.shape)

## popular items must have a rating of 5 using overall cleaned dataset

popular = df_cleaned[df_cleaned['rating']==5]['item_id_new'].value_counts()

## assign rank based on item count
ranking_pop = pd.DataFrame(popular).rename(columns={'item_id_new':\
                                                    'item_count'})
ranking_pop['rank'] = ranking_pop['item_count'].rank(method='dense',\
                                                     ascending=False)
ranking_pop.head()

(190004, 26)


,item_count,rank
168.0,245,1.0
1389.0,223,2.0
66.0,205,3.0
169.0,200,4.0
546.0,195,5.0


In [21]:
def Novelty(topNPredicted):
    n = 0
    total = 0
    for user in topNPredicted.keys():
        for rating in topNPredicted[user]:
            itemID = rating[0]
            rank = ranking_pop.loc[itemID,'rank']
#             print('Item ID: {0} is ranked {1}'.format(itemID,rank))
#             if rank > 259:
#                 print('error')
            total += rank
            n += 1
    return round(total / n, 2)

# print('Novelty score using BaselineOnly: ', Novelty(topNPredicted_BsO))

# print('Novelty score using SVDpp: ',\
#        Novelty(topNPredicted_SVDpp))

print('Novelty score using SVD: ', Novelty(topNPredicted_SVD))

# print('Novelty score using KNNWithZScore: ', Novelty(topNPredicted_KNNZ))

Novelty score using SVD:  84.37


<div class='alert alert-warning'>
#### compare algorithms using KFold and whole data set
- using metrics: `Precision@k` and `Recall@k` functions from Surprise library
<img src="./screenshots/precision_recall.png" width=300 />

- an item is <font color=blue>considered relevant</font> if its <font color=red>true rating $r_{ui}$</font> is <font color=red>greater than a given threshold</font>.
- an item is <font color=blue>considered recommended</font> if its <font color=red>estimated rating $\hat r_{ui}$ is greater than the threshold</font>, and if it is <font color=red>among the k highest estimated ratings</font>.

In [22]:
## use precision@k and recall@k functions from Surprise library

def precision_recall_at_k(predictions, k=10, threshold=5.0):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r>=threshold) and (est>=threshold))
                              for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [23]:
# calculate average precision and recall for SVD

kf = KFold(n_splits=5)

algo = SVD(n_epochs=15,lr_all=0.007,reg_all=0.1)

pmetrics = {}
rmetrics = {}
a=1

for trainset, testset in kf.split(data1):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5,\
                                                threshold=5.0)

    # Precision and recall can then be averaged over all users
    pmetrics[a] = (sum(prec for prec in precisions.values())\
                    / len(precisions))
    rmetrics[a] = (sum(rec for rec in recalls.values()) / len(recalls))
    a +=1

print('average precision for SVD: ',\
      float(sum(pmetrics.values()))/len(pmetrics))

print('average recall for SVD: ',\
      float(sum(rmetrics.values()))/len(rmetrics))

average precision for SVD:  1.0
average recall for SVD:  0.2999380757711686
